In [40]:
from glob import glob
import os
import sys

In [48]:
surf_dir = '/Users/kristianeschenburg/Desktop/Research/Data/Surfaces/'
flow_dir = '/Users/kristianeschenburg/Desktop/Research/Data/FlowLines/'

surfaces = glob('{:}*white*.acpc_dc.surf.gii'.format(surf_dir))

In [50]:
os.chdir('../')

import surfaceflow.flows as flows
from niio import loaded,write
import nibabel as nb
import numpy as np

import pandas as pd

In [51]:
G_DTYPE = np.float64
steps = 10
timestep = 5

nverts = 32492
ncoors = 3

Compute surface flow for surfaces, and save flow as *.dat* file.

In [52]:
for surface_file in surfaces:
    
    sid = surface_file.split('/')[-1].split('.')[0]
    outbase = '{:}{:}.L.white.FlowLines.nb.10.diff.5.acpc'.format(flow_dir,sid)
    
    surface = nb.load(surface_file)
    vertices = surface.darrays[0].data
    triangles = surface.darrays[1].data

    flow_file = '{:}.dat'.format(outbase)
    curvs = flows.flow_trajectories(vertices=vertices, triangles=triangles, 
                                    flow_file=flow_file,
                                    steps=steps, timestep=timestep)

 step 10 on 10 done 
 step 10 on 10 done 
 step 10 on 10 done 
 step 10 on 10 done 
 step 10 on 10 done 
 step 10 on 10 done 
 step 10 on 10 done 
 step 10 on 10 done 
 step 10 on 10 done 
 step 10 on 10 done 


Compute surfaces for each flow iteration.

In [64]:
flow_lines = glob('{:}*.dat'.format(flow_dir))

for fl in flow_lines:
    
    sid = fl.split('/')[-1].split('.')[0]
    outbase = '{:}{:}.L.white.FlowAngle.nb.10.diff.5.acpc'.format(flow_dir,sid)
    
    lines = np.memmap(fl, 
                      dtype=G_DTYPE, 
                      mode='r', 
                      shape=(steps, nverts, ncoors))
    lines = np.asarray(lines).squeeze()
    
    for tg in [2,9]:
        
        outsurf = '{:}{:}.L.white.Flow.{:}.nb.10.diff.5.acpc.surf.gii'.format(flow_dir,sid,tg)
        flow_verts = lines[tg,:,:].squeeze().astype('float32')
        write.save_surf(flow_verts, triangles, outsurf, hemisphere='CortexLeft')
        

Compute flow angles with a surface step size of 2.

In [65]:
for fl in flow_lines:
    
    sid = fl.split('/')[-1].split('.')[0]
    outbase = '{:}{:}.L.white.FlowAngle.nb.10.diff.5.acpc'.format(flow_dir,sid)
    
    lines = np.memmap(fl, 
                      dtype=G_DTYPE, 
                      mode='r', 
                      shape=(steps, nverts, ncoors))
    lines = np.asarray(lines).squeeze()
    
    for tg in [2,9]:
        
        source = lines[tg-2,:,:].squeeze()
        target = lines[tg,:,:].squeeze()
        [r,theta,phi] = flows.flow_angle(source, target)
        
        angles = {'theta': theta, 
                  'phi': phi}
        
        outpath = '{:}.{:}.{:}.csv'.format(outbase,tg-2,tg)
        angles = pd.DataFrame(angles)
        angles.to_csv(path_or_buf=outpath,
                     header=False,index=False,sep='\t')
    
    


In [1]:
from niio import write

In [3]:
write.save_surf??

Signature: write.save_surf(vertices, triangles, output, hemisphere=None)
Source:   
def save_surf(vertices, triangles, output, hemisphere=None):

    """
    Save a list of vertices and triangles to a surface file.

    Parameters:
    - - - - -
    vertices: array, float
        3d coordinates of mesh points
    triangles: array, int
        list of mesh triangles
    output: string
        output file name
    hemisphere: string
        'CortexLeft' or 'CortexRight'
    """

    # Initialize meta-data dictionary structure
    meta_dict = {'AnatomicalStructurePrimary': hemisphere,
                 'AnatomicalStructureSecondary': 'GrayWhite',
                 'Caret-Version': '5.65',
                 'GeometricType': 'Anatomical'}

    # Write meta data to GiftiMetaData object
    meta = nb.gifti.gifti.GiftiMetaData()
    for k, v in meta_dict.items():
        nvp = nb.gifti.gifti.GiftiNVPairs(name=k, value=v)
        meta.data.append(nvp)

    # Initialize GiftiCoordSystem object
    